In [16]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [34]:
from langchain.agents import tool
from langchain.tools import BaseTool
from typing import Optional, Type
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

from pydantic import BaseModel, Field

In [20]:
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [21]:
@tool()
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"


search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [57]:
from langchain.tools import BaseTool
from typing import Optional, Type

from pydantic import BaseModel, Field


class RouteInfo(BaseModel):
    start: str = Field(default="busan",
                       description="Starting location, use busan if not specified")
    end: str = Field(description="Ending location")


class RoutingInput(BaseModel):
    route_object: RouteInfo = Field(
        description="Object containing route information, if start is not specified, use busan")


class RouteTool(BaseTool):
    name = "get_route_from_start_to_end"
    description = "Get route from start to end"

    def _run(self, route_object: RoutingInput):
        # print("i'm running")
        print(route_object)
        return route_object['start'] + " to " + route_object['end']
    args_schema: Optional[Type[BaseModel]] = RoutingInput

In [60]:
import langchain
langchain.debug = True
tools = [RouteTool()]

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini-0613")
open_ai_agent = initialize_agent(tools,
                                 llm,
                                 agent=AgentType.OPENAI_FUNCTIONS,
                                 verbose=True)
open_ai_agent.run("I'm at busan right now. How to get from seoul to Boston?")

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "I'm at busan right now. How to get from seoul to Boston?"
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: You are a helpful AI assistant.\nHuman: I'm at busan right now. How to get from seoul to Boston?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [1.28s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "get_route_from_start_to_end",
                "arguments": "{\n  \"rou

'To get from Seoul to Boston, you can take the following route:\n\n1. Fly from Seoul to Boston. There are several airlines that operate flights between these two cities, including Korean Air, Asiana Airlines, and Delta Air Lines. The flight duration is approximately 13-15 hours, depending on the layovers.\n\nPlease note that this information is subject to change and it is recommended to check with the airlines for the most up-to-date schedules and fares.'

In [37]:
RouteTool().args

{'route_object': {'allOf': [{'$ref': '#/$defs/RouteInfo'}],
  'description': 'Object containing route information'}}

In [ ]:
llm